# Controllable generation via RL about text-guided voice conversion


In [1]:
import torch
from datasets import load_from_disk
from vc.encodec_model.nar_bart_model import NARBartForConditionalGeneration
from transformers import AutoTokenizer, BartForConditionalGeneration
import sys
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, AutoModelForSeq2SeqLMWithValueHead, create_reference_model
from tqdm import tqdm

# load the model
ar_checkpoint = "lca0503/speech-chatgpt-base-ar-v2-epoch10-wotrans"
nar_checkpoint = "lca0503/speech-chatgpt-base-nar-v2-epoch4-wotrans"

device = "cuda" if torch.cuda.is_available() else "cpu"
# model = AutoModelForCausalLMWithValueHead.from_pretrained(ar_checkpoint)
# model = BartForConditionalGeneration.from_pretrained(ar_checkpoint)
model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(ar_checkpoint)
model_ref = create_reference_model(model)
tokenizer = AutoTokenizer.from_pretrained(ar_checkpoint)
nar_tokenizer = AutoTokenizer.from_pretrained(nar_checkpoint)
nar_model = NARBartForConditionalGeneration.from_pretrained(nar_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

2024-05-15 10:26:21.089444: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 10:26:22.826267: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from datetime import datetime
import os

now = datetime.now()
ts = now.strftime("%m%d-%H%M")
print("timestamp:", ts)

# define the path
base_path = "/work/b0990106x/trl"
agent_input_dir = f"{base_path}/data-encodec"
agent_output_dir = f"{base_path}/output/{ts}"
env_input_dir = agent_output_dir
env_output_dir = agent_input_dir

if not os.path.exists(agent_output_dir):
    os.makedirs(agent_output_dir)

timestamp: 0515-1026


In [3]:
# load the dataset
dataset = load_from_disk(agent_input_dir)

In [4]:
all_src_encodec_layers = []
all_src_encodec = []
all_instruction = []
# all_instruction_ids = []

layer_len = 8
data_len = 3
# data_len = len(dataset)
print("data_len:", data_len)

for i in range(layer_len):
    all_src_encodec_layers.append(dataset[f"src_encodec_{i}"])

for i in range(data_len):
    src_encodec = []
    for j in range(layer_len):
        src_encodec.append(all_src_encodec_layers[j][i])
    all_src_encodec.append(src_encodec)

    all_instruction.append(dataset["instruction"][i])
    # all_instruction_ids.append(ar_tokenizer(all_instruction[i])["input_ids"][1 : -1])

data_len: 3


In [5]:
# print the length of all src encodec
for i in range(data_len):
    print(f"src_encodec_{i} len:", len(all_src_encodec[i][0]))

src_encodec_0 len: 327
src_encodec_1 len: 336
src_encodec_2 len: 131


### Debugging Section

In [6]:
observation_list = []
for i in range(data_len):
    observation_list.append(
        {
            "input": "",
            "src_encodec": all_src_encodec[i],
            "instruction": all_instruction[i],
        }
    )

# # pop the first one
observation_list.pop(0)
all_instruction.pop(0)
observation_list.pop(0)
all_instruction.pop(0)
print("observation_list:", observation_list)
print("all_instruction:", all_instruction)


# for i in range(data_len):
#     observation_list.append({'input': "", 'src_encodec': all_src_encodec[i], 'instruction': all_instruction[i]})

observation_list: [{'input': '', 'src_encodec': [[835, 339, 999, 629, 604, 462, 314, 600, 846, 562, 846, 358, 984, 393, 182, 453, 584, 535, 407, 1021, 701, 843, 945, 495, 563, 495, 495, 727, 317, 604, 475, 835, 835, 835, 339, 475, 339, 123, 254, 103, 561, 858, 646, 755, 375, 548, 435, 233, 323, 395, 819, 475, 339, 835, 779, 257, 339, 341, 170, 38, 38, 103, 408, 62, 141, 731, 73, 651, 143, 875, 321, 310, 310, 972, 679, 582, 808, 813, 808, 291, 722, 982, 627, 192, 764, 531, 291, 466, 567, 601, 771, 112, 688, 348, 793, 793, 11, 192, 23, 983, 1022, 23, 73, 73, 276, 537, 103, 53, 148, 148, 148, 463, 176, 148, 463, 463, 463, 463, 463, 463, 463, 433, 25, 472, 257, 228, 395, 133, 395, 475, 126], [646, 841, 168, 1023, 277, 820, 278, 215, 58, 592, 607, 607, 349, 346, 504, 632, 482, 14, 968, 588, 529, 904, 662, 662, 602, 1013, 662, 386, 617, 870, 648, 1023, 277, 277, 913, 200, 1007, 503, 807, 144, 132, 558, 984, 164, 610, 66, 830, 925, 744, 129, 87, 648, 391, 646, 424, 700, 646, 713, 702, 443, 4,

In [7]:
import sys
sys.path.append("/work/b0990106x/TextRL/vc")
from vc.trainer_encodec_vc_inference import get_ar_prediction
from types import SimpleNamespace


args_predict = SimpleNamespace(output_path=f"{base_path}/output/{ts}/example.wav", seed=0, device=device)

decode_ar = get_ar_prediction(args_predict, model, nar_model, tokenizer, nar_tokenizer, all_src_encodec[0], all_instruction[0], 0)

decode_ar_str = tokenizer.convert_tokens_to_string(
                [f"v_tok_{u}" for u in decode_ar]
            )

print(decode_ar_str)

Decode AR:  tensor([[    2,     0, 51100, 51100, 51063, 50633, 51111, 50800, 50800, 51002,
         51002, 50642, 50495, 50364, 50821, 51151, 50364, 50676, 50680, 50643,
         51202, 51245, 50676, 50676, 51171, 50798, 51199, 50312, 50703, 50703,
         50753, 50673, 50371, 50604, 50744, 50744, 50744, 50698, 50416, 50416,
         51171, 50767, 51171, 50689, 50473, 50802, 50802, 50338, 50948, 51012,
         51012, 50858, 51012, 50650, 50590, 50756, 51287, 50498, 50858, 51019,
         50686, 50427, 51273, 51232, 50317, 50660, 50660, 50522, 51169, 50317,
         50574, 50700, 50513, 51019, 51040, 51262, 51262, 50601, 50813, 50487,
         50789, 50406, 50744, 51205, 50541, 50321, 50673, 51100, 50740, 50471,
         51044, 50838, 51196, 50863, 50276, 50285, 51215, 50634, 50271, 50731,
         50981, 51106, 50347, 50634, 50553, 50821, 51168, 50821, 50657, 51186,
         51241, 51100, 50368, 50290, 50386, 51100, 51100, 51100, 50604, 50604,
         51225, 50653, 50582, 50406, 505

In [8]:
from datetime import datetime
import os

now = datetime.now()
ts = now.strftime("%m%d-%H%M")
print("timestamp:", ts)
log_dir = f"logs/{ts}"
os.makedirs(log_dir, exist_ok=True)

lr= 0.0000141
batch_size = 1
mini_batch_size = 1


ppo_config = PPOConfig(batch_size=1, mini_batch_size=1, log_with='tensorboard', learning_rate=lr, project_kwargs={'logging_dir': log_dir})
ppo_trainer = PPOTrainer(config = ppo_config, model = model, ref_model=model_ref, tokenizer=tokenizer)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


timestamp: 0515-1026


/work/b0990106x/trl/trl/trainer/ppo_trainer.py:262: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [9]:
from importlib import reload
from NISQA.nisqa.NISQA_model import nisqaModel

import sys
sys.path.append("/work/b0990106x/trl/vc") 
from vc.trainer_encodec_vc_inference import get_ar_prediction_v2


def get_reward(predicted_list, single_src_encodec, single_instruction, episode_counter,finish):
    reward = 0
    # predicted_list will be one text of "v_tok_410v_tok_411v_tok_595 ...""
    # predicted_token will be a list of [v_tok_410, v_tok_411, v_tok_595 ...]
    
    if finish or len(predicted_list) >= 1000:
        try:
            # predicted_tokens = predicted_list[0][1:-1]
            predicted_tokens = [f'v_tok_{u}' for u in predicted_list.split("v_tok_")[1:]]
            predicted_ids = tokenizer.convert_tokens_to_ids([f"{u}" for u in predicted_tokens])
            print("predict length: ", len(predicted_ids))
            print("predicted_tokens: ", predicted_tokens)
            print("predicted_ids: ", predicted_ids)

            decode_ar = get_ar_prediction_v2(
                args_predict,
                predicted_ids,
                nar_model,
                tokenizer,
                nar_tokenizer,
                single_src_encodec,
                single_instruction,
                episode_counter,
            )
            # print("decode_ar:", decode_ar)
            
            # use nisqa to get the reward
            args_nisqa = {
                "mode": "predict_file",
                "pretrained_model": f"{base_path}/NISQA/weights/nisqa.tar",
                "deg": f"{base_path}/output/{ts}/example.wav",
                "data_dir": None,
                "output_dir": f"{base_path}/NISQA/result/",
                "csv_file": None,
                "csv_deg": None,
                "num_workers": 0,
                "bs": 1,
                "ms_channel": None,
            }
            args_nisqa["tr_bs_val"] = args_nisqa["bs"]
            args_nisqa["tr_num_workers"] = args_nisqa["num_workers"]

            nisqa = nisqaModel(args_nisqa)
            prediction = nisqa.predict()
            reward = float(prediction["mos_pred"].iloc[0])*10
            # reward = float(prediction["mos_pred"].iloc[0])-3.0
            print(
                "Length of predicted_list:",
                len(predicted_list),
                ", Reward:",
                reward,
            )

        except Exception as e:
            print("Error:", e)
            reward = 0

    return reward

In [10]:
import logging
import os
import sys

output_log_path = f"logs/log_{ts}.log"

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

handlers = logger.handlers[:]
for handler in handlers:
    logger.removeHandler(handler)

file_handler = logging.FileHandler(output_log_path)
logger.addHandler(file_handler)

In [11]:
import time
from trl.core import respond_to_batch

start_time = time.time()

try:
    for iteration in tqdm(range(100)):
        query_txt = decode_ar_str
        query_tensor = tokenizer.encode(query_txt, return_tensors="pt")
        query_tensor = query_tensor.to(device)
        
        # FILEPATH: /work/b0990106x/trl/textrl.ipynb
        response_tensor = respond_to_batch(model, query_tensor, txt_len=500)
        # response_tensor = model.generate(query_tensor)
        response_text = tokenizer.decode(response_tensor[0], skip_special_tokens=True)
        # Mimic batch structure
        batch = {
            "query": query_tensor,
            "response": response_text
        }
        reward_float = get_reward(response_text, all_src_encodec[0], all_instruction[0], iteration, True)
        reward_length = len(tokenizer.decode(response_tensor[0], skip_special_tokens=True))
        reward = torch.tensor([float(reward_float)], device=device)
        train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], [reward])
        ppo_trainer.log_stats(train_stats, batch, reward)

        print(f"Iteration {iteration + 1}, Reward: {reward.item()}, Length: {len(response_tensor[0])}, Reward_Length: {reward_length}, Predicted Text: {response_text}")

except Exception as e:
    print("An error occurred:", e)

print("used time: ", time.time() - start_time)

  0%|          | 0/100 [00:00<?, ?it/s]

predict length:  393
predicted_tokens:  ['v_tok_753', 'v_tok_353', 'v_tok_619', 'v_tok_325', 'v_tok_976', 'v_tok_73', 'v_tok_133', 'v_tok_20', 'v_tok_779', 'v_tok_233', 'v_tok_629', 'v_tok_593', 'v_tok_475', 'v_tok_99', 'v_tok_904', 'v_tok_421', 'v_tok_216', 'v_tok_435', 'v_tok_234', 'v_tok_213', 'v_tok_112', 'v_tok_358', 'v_tok_754', 'v_tok_435', 'v_tok_846', 'v_tok_437', 'v_tok_358', 'v_tok_739', 'v_tok_951', 'v_tok_593', 'v_tok_213', 'v_tok_524', 'v_tok_951', 'v_tok_233', 'v_tok_479', 'v_tok_479', 'v_tok_504', 'v_tok_1023', 'v_tok_803', 'v_tok_317', 'v_tok_940', 'v_tok_1022', 'v_tok_151', 'v_tok_257', 'v_tok_106', 'v_tok_904', 'v_tok_395', 'v_tok_904', 'v_tok_904', 'v_tok_661', 'v_tok_99', 'v_tok_309', 'v_tok_435', 'v_tok_248', 'v_tok_248', 'v_tok_754', 'v_tok_997', 'v_tok_997', 'v_tok_997', 'v_tok_358', 'v_tok_830', 'v_tok_233', 'v_tok_951', 'v_tok_146', 'v_tok_411', 'v_tok_172', 'v_tok_535', 'v_tok_141', 'v_tok_721', 'v_tok_317', 'v_tok_317', 'v_tok_151', 'v_tok_59', 'v_tok_56', '

/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/b0990106x/miniconda3/envs/trl/lib/python3.10/site-packages/librosa/feature/spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Length of predicted_list: 3448 , Reward: 22.342751026153564


/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (22.12) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (1005.08) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1222: UserWarning: The average ratio of batch (2882.53) exceeds threshold 10.00. Skipping batch.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1285: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std_scores = data["scores"].std()
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1312: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered 

...

Length of predicted_list: 4411 , Reward: 12.880290746688843


/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1285: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  std_scores = data["scores"].std()
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -99.38 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
/work/b0990106x/trl/trl/trainer/ppo_trainer.py:1312: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at ../aten/src/ATen/native/ReduceOps.cpp:1807.)
  stats["tokens/queries_len_std"] = torch.std(query_lens)

Iteration 78, Reward: 12.880290985107422, Length: 500, Reward_Length: 4411, Predicted Text: v_tok_835v_tok_998v_tok_215v_tok_481v_tok_567v_tok_716v_tok_716v_tok_980v_tok_980v_tok_378v_tok_808v_tok_326v_tok_980v_tok_980v_tok_998v_tok_74v_tok_74v_tok_980v_tok_980v_tok_74v_tok_980v_tok_491v_tok_699v_tok_699v_tok_699v_tok_976v_tok_699v_tok_411v_tok_181v_tok_411v_tok_466v_tok_650v_tok_43v_tok_1001v_tok_871v_tok_840v_tok_411v_tok_820v_tok_980v_tok_601v_tok_74v_tok_269v_tok_812v_tok_362v_tok_248v_tok_532v_tok_248v_tok_870v_tok_567v_tok_567v_tok_358v_tok_532v_tok_723v_tok_532v_tok_378v_tok_532v_tok_532v_tok_755v_tok_584v_tok_504v_tok_723v_tok_504v_tok_532v_tok_532v_tok_532v_tok_948v_tok_961v_tok_796v_tok_456v_tok_807v_tok_807v_tok_807v_tok_385v_tok_385v_tok_840v_tok_840v_tok_840v_tok_840v_tok_840v_tok_382v_tok_382v_tok_382v_tok_382v_tok_382v_tok_382v_tok_382v_tok_382v_tok_382v_tok_816v_tok_382v_tok_816v_tok_382v_tok_980v_tok_382v_tok_74v_tok_840v_tok_57v_tok_363v_tok_407v_tok_724v_tok_228v_tok

 78%|███████▊  | 78/100 [17:05<04:49, 13.15s/it]


KeyboardInterrupt: 